# ASEdb-SQLite3 backend example

In [3]:
import numpy as np
from ase.lattice import bulk
from ase.calculators.singlepoint import SinglePointCalculator
from ase.db.row import atoms2dict

from abcd.structurebox import StructureBox
from abcd.authentication import Credentials
import asedb_sqlite3_backend.asedb_sqlite3_backend as asedb_sqlite3_backend

###Establish the connection to the backend, generate random configurations and insert them to the database.

In [4]:
!rm ~/databases/all/test.db
box = StructureBox(asedb_sqlite3_backend.ASEdbSQlite3Backend(database='test.db', user=None))
token = box.authenticate(Credentials(None))

N = 10
for i in range(N):
    atoms = bulk('Si', crystalstructure='diamond', a=5.43, cubic=True)
    atoms.rattle()

    # simulate a calculation with random results
    e = np.random.uniform()
    f = np.random.uniform(size=3*len(atoms)).reshape((len(atoms), 3))
    s = np.random.uniform(size=9).reshape((3, 3))
    calc = SinglePointCalculator(atoms, energy=e, forces=f, stress=s)
    atoms.set_calculator(calc)
    f = atoms.get_forces()
    e = atoms.get_potential_energy()

    # add some arbitrary data
    atoms.info['integer_info'] = 42
    atoms.info['real_info'] = 217
    atoms.info['config_type'] = 'diamond'
    atoms.new_array('array_data', np.ones_like(atoms.numbers))
    
    box.insert(token, atoms)

### Use ase-db to display the created database

In [5]:
!ase-db ~/databases/all/test.db -c ++

id|age|user   |formula|calculator|energy| fmax|pbc| volume|charge|   mass| smax|real_info|integer_info|config_type
 1| 2s|patrick|Si8    |unknown   | 0.422|1.495|TTT|160.103| 0.000|224.684|0.860|      217|          42|diamond    
 2| 2s|patrick|Si8    |unknown   | 0.045|1.340|TTT|160.103| 0.000|224.684|0.890|      217|          42|diamond    
 3| 2s|patrick|Si8    |unknown   | 0.956|1.220|TTT|160.103| 0.000|224.684|0.791|      217|          42|diamond    
 4| 2s|patrick|Si8    |unknown   | 0.212|1.153|TTT|160.103| 0.000|224.684|0.963|      217|          42|diamond    
 5| 2s|patrick|Si8    |unknown   | 0.949|1.189|TTT|160.103| 0.000|224.684|0.930|      217|          42|diamond    
 6| 2s|patrick|Si8    |unknown   | 0.065|1.272|TTT|160.103| 0.000|224.684|0.908|      217|          42|diamond    
 7| 2s|patrick|Si8    |unknown   | 0.914|1.488|TTT|160.103| 0.000|224.684|0.985|      217|          42|diamond    
 8| 2s|patrick|Si8    |unknown   | 0.720|1.468|TTT|160.103| 0.000|224.684|0.939|

###Query the database and get Atoms objects back

In [6]:
count = 0
for i in box.find(token, 'energy>0.5'):
    print(i)
    count += 1
print '\nFound', count, 'structures'

Atoms(symbols='Si8', array_data=..., positions=..., cell=[5.43, 5.43, 5.43], pbc=[True, True, True], calculator=SinglePointCalculator(...))
Atoms(symbols='Si8', array_data=..., positions=..., cell=[5.43, 5.43, 5.43], pbc=[True, True, True], calculator=SinglePointCalculator(...))
Atoms(symbols='Si8', array_data=..., positions=..., cell=[5.43, 5.43, 5.43], pbc=[True, True, True], calculator=SinglePointCalculator(...))
Atoms(symbols='Si8', array_data=..., positions=..., cell=[5.43, 5.43, 5.43], pbc=[True, True, True], calculator=SinglePointCalculator(...))
Atoms(symbols='Si8', array_data=..., positions=..., cell=[5.43, 5.43, 5.43], pbc=[True, True, True], calculator=SinglePointCalculator(...))

Found 5 structures


### Copy all atoms with id greater than 7

In [7]:
# Copy all Atoms structures with id greater than 7
atoms_it = box.find(token, 'id>7')
atoms = [at for at in atoms_it]
print 'Copied', len(atoms), 'structures'

Copied 3 structures


### Delete them from the database

In [8]:
res = box.remove(token, 'id>7', just_one=False)
print res.msg

Deleted 3 rows


In [9]:
!ase-db ~/databases/all/test.db -c ++

id|age|user   |formula|calculator|energy| fmax|pbc| volume|charge|   mass| smax|real_info|integer_info|config_type
 1| 5s|patrick|Si8    |unknown   | 0.422|1.495|TTT|160.103| 0.000|224.684|0.860|      217|          42|diamond    
 2| 5s|patrick|Si8    |unknown   | 0.045|1.340|TTT|160.103| 0.000|224.684|0.890|      217|          42|diamond    
 3| 5s|patrick|Si8    |unknown   | 0.956|1.220|TTT|160.103| 0.000|224.684|0.791|      217|          42|diamond    
 4| 5s|patrick|Si8    |unknown   | 0.212|1.153|TTT|160.103| 0.000|224.684|0.963|      217|          42|diamond    
 5| 5s|patrick|Si8    |unknown   | 0.949|1.189|TTT|160.103| 0.000|224.684|0.930|      217|          42|diamond    
 6| 5s|patrick|Si8    |unknown   | 0.065|1.272|TTT|160.103| 0.000|224.684|0.908|      217|          42|diamond    
 7| 5s|patrick|Si8    |unknown   | 0.914|1.488|TTT|160.103| 0.000|224.684|0.985|      217|          42|diamond    
Rows: 7
Keys: config_type, integer_info, real_info


### Insert the copies back into the database

In [10]:
res = box.insert(token, atoms)
print res.msg, 'at ids', res.inserted_ids

Inserted 3 configurations at ids [11, 12, 13]


In [11]:
!ase-db ~/databases/all/test.db -c ++

id|age|user   |formula|calculator|energy| fmax|pbc| volume|charge|   mass| smax|real_info|integer_info|config_type
 1| 7s|patrick|Si8    |unknown   | 0.422|1.495|TTT|160.103| 0.000|224.684|0.860|      217|          42|diamond    
 2| 7s|patrick|Si8    |unknown   | 0.045|1.340|TTT|160.103| 0.000|224.684|0.890|      217|          42|diamond    
 3| 7s|patrick|Si8    |unknown   | 0.956|1.220|TTT|160.103| 0.000|224.684|0.791|      217|          42|diamond    
 4| 7s|patrick|Si8    |unknown   | 0.212|1.153|TTT|160.103| 0.000|224.684|0.963|      217|          42|diamond    
 5| 7s|patrick|Si8    |unknown   | 0.949|1.189|TTT|160.103| 0.000|224.684|0.930|      217|          42|diamond    
 6| 7s|patrick|Si8    |unknown   | 0.065|1.272|TTT|160.103| 0.000|224.684|0.908|      217|          42|diamond    
 7| 7s|patrick|Si8    |unknown   | 0.914|1.488|TTT|160.103| 0.000|224.684|0.985|      217|          42|diamond    
11| 1s|patrick|Si8    |unknown   | 0.720|1.468|TTT|160.103| 0.000|224.684|0.939|